# Requester's Guide 

The current release of Ravenverse provides a collection of easy to use libraries that allow requesters to build mathematical algorithms or models and compute these operations by distributing them across multiple Providers. This provides an increase in speed and efficiency when dealing with a large number of mathematical operations.

Distributed Computing is the linking of various computing resources like PCs and smartphones to share and coordinate their processing power for a common computational requirement, such as the training of a large Machine Learning model. These resources or nodes communicate with a central server and in some cases with each other, such that each node receives some data and completes a subset of a task. These nodes can coordinate their computations to complete a large and complex computational requirement in a fast and efficient manner.

This tutorial explains how to build a distributed graph that the requester can compile and execute via the participating Provider nodes present on the Ravenverse network. 

### Installing Dependencies


In [ ]:
!pip install ravop
!pip install ravdl

### Ravenverse Token

The requester must connect to the Ravenverse using a unique token that they can generate by logging into Raven's Website (https://www.ravenverse.ai/) using their MetaMask wallet credentials.

### Setting Environment Variables

In [16]:
%env TOKEN=YOUR_TOKEN
%env RAVENVERSE_URL=http://server.ravenverse.ai
%env RAVENVERSE_FTP_HOST=server.ravenverse.ai
%env RAVENVERSE_FTP_URL=server.ravenverse.ai

env: TOKEN=YOUR_TOKEN
env: RAVENVERSE_URL=http://server.ravenverse.ai
env: RAVENVERSE_FTP_HOST=server.ravenverse.ai
env: RAVENVERSE_FTP_URL=server.ravenverse.ai


### Generating a TorchScript Model File

Let's start off by generating a ```.pt``` file for a regular CNN model.

In [2]:
import torch
import torch.nn as nn

class Net(nn.Module):    
    def __init__(self):
        super(Net, self).__init__()
        self.conv2d_1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=(3, 3), padding='same')
        self.act_1 = nn.ReLU()
        self.maxpool2d_1 = nn.MaxPool2d(kernel_size=(2, 2), stride=2)
        self.drp_1 = nn.Dropout(0.25)
        self.bn_1 = nn.BatchNorm2d(16)
        self.maxpool2d_2 = nn.MaxPool2d(kernel_size=(2, 2), stride=2)
        self.conv2d_2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3, 3), padding='same')
        self.act_2 = nn.ReLU()
        self.maxpool2d_3 = nn.MaxPool2d(kernel_size=(2, 2), stride=2)
        self.drp_2 = nn.Dropout(0.25)
        self.bn_2 = nn.BatchNorm2d(32)
        self.flatten = nn.Flatten()
        self.dense_1 = nn.Linear(in_features=32,out_features=256)
        self.act_3 = nn.ReLU()
        self.drp_3 = nn.Dropout(0.4)
        self.bn_3 = nn.BatchNorm1d(256)
        self.dense_2 = nn.Linear(in_features=256, out_features=10)
        self.act_4 = nn.Softmax(dim=-1)

    def forward(self, x):
        out = self.conv2d_1(x)
        out = self.act_1(out)
        out = self.maxpool2d_1(out)
        out = self.drp_1(out)
        out = self.bn_1(out)
        out = self.maxpool2d_2(out)
        out = self.conv2d_2(out)
        out = self.act_2(out)
        out = self.maxpool2d_3(out)
        out = self.drp_2(out)
        out = self.bn_2(out)
        out = self.flatten(out)
        out = self.dense_1(out)
        out = self.act_3(out)
        out = self.drp_3(out)
        out = self.bn_3(out)
        out = self.dense_2(out)
        out = self.act_4(out)
        return out


In [3]:
model = Net()

model_script = torch.jit.script(model)

model_script.save('test_model.pt')

Running the above cell will result in the creation of a ```test_model.pt``` file.

### Importing Dependencies

In [4]:
import os
import ravop as R
import numpy as np
from ravdl.v2 import Pytorch_Model
from ravdl.v2.optimizers import Adam
from ravdl.v2.utils.data_manipulation import batch_iterator

from sklearn.model_selection import train_test_split
from sklearn import datasets
import torch

Checking version of RavDL...
Current version of ravdl is 0.12
Latest version of ravdl is 0.12


In [5]:
R.initialize(ravenverse_token=os.environ.get("TOKEN")) # Initialize with Requester token

R.flush()             # Flush and discard existing graphs associated to this Requester (if any)

R.Graph(name='cnn_model', algorithm='convolutional_neural_network', approach='distributed')     # create the distributed compute graph

2023-06-12 16:33:29,370 [MainThread  ] [DEBUG]  Checking version of Ravop...
2023-06-12 16:33:29,581 [MainThread  ] [DEBUG]  Initializing...
2023-06-12 16:33:29,582 [MainThread  ] [DEBUG]  Creating FTP developer credentials...
2023-06-12 16:33:32,142 [MainThread  ] [DEBUG]  Error in speedtest:HTTP Error 403: Forbidden
2023-06-12 16:33:32,144 [MainThread  ] [DEBUG]  FTP Upload Blocksize:8192
2023-06-12 16:33:32,145 [MainThread  ] [DEBUG]  FTP User credentials:server.ravenverse.ai 6007485310 JX44JOJKUE
2023-06-12 16:33:32,317 [MainThread  ] [DEBUG]  Initialized Successfully!
2023-06-12 16:33:32,419 [MainThread  ] [DEBUG]  Your Current Graph ID:229
2023-06-12 16:33:32,675 [MainThread  ] [DEBUG]  
2023-06-12 16:33:32,677 [MainThread  ] [DEBUG]  {'message': 'Requester flushed'}


### Dataset Preprocessing

In [6]:
def to_categorical(x, n_col=None):
    if not n_col:
        n_col = np.amax(x) + 1
    one_hot = np.zeros((x.shape[0], n_col))
    one_hot[np.arange(x.shape[0]), x] = 1
    return one_hot

data = datasets.load_digits()
X = data.data
y = data.target

# Convert to one-hot encoding
y = to_categorical(y.astype("int"))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=1)

# Reshape X to (n_samples, channels, height, width)
X_train = X_train.reshape((-1, 1, 8, 8))
X_test = X_test.reshape((-1, 1, 8, 8))

In [7]:
model_op = R.model('test_model.pt')
optimizer = Adam()

model = Pytorch_Model(model_op=model_op)
model.initialize(optimizer)

Model path:  test_model.pt True
Model exists
Id self:  1


In [8]:
epochs = 10

### Training Loop

In [9]:
for i in range(epochs):
    for X_batch, y_batch in batch_iterator(X_train, y_train, batch_size=256):
        X_t = R.t(X_batch.astype(np.float32))
        y_t = R.t(y_batch.astype(np.float32))

        out = model._forward_pass(X_t)
        loss = R.square_loss(y_t, out)

        # Set step = True whenever optimizer step needs to be called after backprop (defaults to True).
        model._backward_pass(loss, step = True)

### Saving the Model for Fetching it Post-Execution.

In [10]:
model.save_model(name='my_net')

2023-06-12 16:33:36,246 [MainThread  ] [DEBUG]  
2023-06-12 16:33:36,247 [MainThread  ] [DEBUG]  Persisting Op: my_net


### Saving a Prediction output for Post-Execution

Persisted a model forward pass output by the name "output". We will fetch this result later post-execution.
Persisting Ops are a special category of Ops that stay in the ravenverse once the graph gets executed. The requester must explicitly mention which ops they want to save in their code. It is a good idea to write the code such that persisting ops contain the relevant results (in this case, variable - ```output```).

***Note:*** Make sure that the ```name``` parameter for each persisting Op is unique within a graph so that later it can be retrieved.

In [11]:
test_input = R.t(X_test.astype(np.float32))

output = model._forward_pass(test_input, training=False)
output.persist_op(name="output")

2023-06-12 16:33:40,210 [MainThread  ] [DEBUG]  
2023-06-12 16:33:40,212 [MainThread  ] [DEBUG]  Persisting Op: output


### Activating the Graph

This step compiles all the Operations (Ops) in the Graph and shows the Cost of Execution (in Raven Tokens) and also the number of participant Provider nodes that will be required. This step makes the graph ready for execution. No more Ops can be added to the graph after this.


In [12]:
R.activate()

2023-06-12 16:33:56,120 [MainThread  ] [DEBUG]  {'message': 'Ops Persisted Successfully!'}
2023-06-12 16:34:01,260 [MainThread  ] [DEBUG]  

2023-06-12 16:34:01,263 [MainThread  ] [DEBUG]  Graph Compiled Successfully. Ready to Execute!
2023-06-12 16:34:01,264 [MainThread  ] [DEBUG]  Cost: 7455.57 RAVEN TOKENS
2023-06-12 16:34:01,266 [MainThread  ] [DEBUG]  Max Participants: 1


'Graph Compiled Successfully. Ready to Execute!'

### Executing the Graph and Tracking Progress

In [13]:
R.execute()
R.track_progress()

2023-06-12 16:34:05,872 [MainThread  ] [DEBUG]  Graph Execute Initiated


Progress |████████████████████████████████████████| 100/100 [100%] in 1:48.6 (0.92/s) 

Graph Computed Successfully!


### Fetching the Persisted Op to Calculate Model Accuracy

In [14]:
prediction = R.fetch_persisting_op(op_name="output")       # output was the name of the persisted op
y_pred = np.argmax(prediction.detach().numpy(), axis=-1)
y_test = np.argmax(y_test, axis=-1)

accuracy = np.sum(y_pred == y_test, axis=0) / len(y_test)

print("Accuracy:", accuracy)

Accuracy: 0.741307371349096


### Fetching the Trained Model to Calculate Accuracy 

In [15]:
my_net = R.fetch_persisting_op(op_name="my_net")
my_net.eval()
out = my_net(torch.tensor(X_test.astype(np.float32)))
y_pred = np.argmax(out.detach().numpy(), axis=-1)
print(y_pred)
accuracy = np.sum(y_pred == y_test, axis=0) / len(y_test)

print("Accuracy of loaded model:", accuracy)

[1 5 0 7 1 0 6 1 5 4 0 2 2 8 1 6 7 7 7 4 7 1 1 6 0 7 6 1 3 7 0 3 5 3 2 1 8
 1 1 0 7 1 0 0 8 7 2 7 1 3 4 3 4 0 4 7 0 6 7 5 2 1 7 0 0 1 2 3 3 4 0 0 7 4
 7 4 2 1 7 0 1 5 3 4 1 5 5 2 5 2 2 1 2 7 0 8 1 0 4 1 3 8 3 0 2 0 2 0 2 2 3
 2 6 1 1 5 1 2 0 4 1 5 4 4 7 6 7 6 6 1 7 5 6 2 0 3 7 1 1 5 3 4 7 8 5 0 6 0
 6 3 7 6 5 6 2 2 2 3 0 0 6 5 6 4 1 0 6 0 6 4 0 1 1 8 1 2 7 1 1 0 7 6 2 5 3
 6 3 4 6 3 3 7 4 7 2 7 6 1 6 0 4 0 3 1 0 0 7 9 0 1 1 6 8 0 7 5 0 8 2 2 5 2
 0 0 7 4 0 0 3 0 6 3 2 3 5 1 6 0 0 4 2 2 7 3 1 6 7 6 3 0 1 0 2 4 0 6 4 8 5
 5 6 3 1 4 0 4 4 7 7 7 1 5 2 7 0 0 0 4 4 0 1 4 6 4 2 7 5 0 1 6 0 1 1 2 0 0
 5 6 7 0 4 0 0 1 4 7 1 7 0 6 6 8 0 2 2 6 0 0 7 5 1 7 6 4 6 1 0 4 7 1 6 7 8
 1 6 0 8 3 2 4 0 7 6 5 6 9 5 1 5 0 0 4 9 0 0 4 0 4 2 5 4 7 6 4 2 6 0 0 5 6
 7 1 9 2 0 1 0 0 1 7 7 0 6 9 3 2 2 2 0 0 7 0 1 2 0 3 2 1 1 9 7 3 0 0 7 3 1
 7 3 2 7 1 0 4 5 4 1 7 3 6 5 4 0 0 5 9 1 4 5 0 4 3 4 2 7 0 0 0 7 4 6 0 4 5
 7 7 2 7 8 5 2 6 6 7 1 0 1 4 8 0 5 4 1 2 5 7 3 3 0 1 6 7 6 8 3 6 2 5 2 6 4
 5 4 4 0 7 3 1 0 1 6 9 3 